In [11]:
import pandas as pd 
import numpy as np 
 
import recordlinker
from recordlinker import preprocess 

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore', 'info')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
iowa_matches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_matches.csv')
iowa_nonmatches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_nonmatches.csv')

In [4]:
iowa_matches.head()

,uid1915,fname1915,lname1915,fullname1915,yob1915,hhid,fname1940,lname1940,fullname1940,yob1940,uid-hhid
0,uid0910071227,donald d,cutler,donald d cutler,1911,19067,donald dean,cutler,donald dean cutler,1911,uid0910071227-19067
1,uid0063131339,homer,taylor,homer taylor,1912,71505,homer ellis,taylor,homer ellis taylor,1912,uid0063131339-71505
2,uid0044088276,earl,stearnes,earl stearnes,1899,109708,earl,stearns,earl stearns,1900,uid0044088276-109708
3,uid0067053130,theodore,hornaday,theodore hornaday,1904,108304,theodore i,harnaday,theodore i harnaday,1904,uid0067053130-108304
4,uid0066046148,jack r,turner,jack r turner,1907,105092,jack r,turner,jack r turner,1907,uid0066046148-105092


In [18]:
union_matches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/unionarmy_matches.csv')

In [19]:
union_matches.head()

,recidnum,recname1,recname2,last1,first1,last2,first2
0,100501001,anson charles h,anson charles h,anson,charles h,anson,charles h
1,100501002,allsheskey theodore f,allsheskey theodore f,allsheskey,theodore f,allsheskey,theodore f
2,100501003,bill charles w,bill c w,bill,charles w,bill,c w
3,100501004,bradley george a,bradley george a,bradley,george a,bradley,george a
4,100501005,bunitt william n,burritt william n,bunitt,william n,burritt,william n


### String Embedding Example

In [5]:
from recordlinker.preprocess import embed_letters, embed_shingles, disembed_letters, disembed_shingles

name = 'kailin lu'
max_length = 12 

print('Embed Letters: \n', 
      embed_letters(name, max_length), 
      disembed_letters(embed_letters(name, max_length))) 

print('Embed Letters Normalized: \n', 
      embed_letters(name, max_length, normalize=True), 
      disembed_letters(embed_letters(name, max_length, normalize=True)))

print('Embed 2-Shingles: \n', 
      embed_shingles(name, max_length), 
      disembed_shingles(embed_shingles(name, max_length))) 

print('Embed 2-Shingles Normalized: \n', 
      embed_shingles(name, max_length, normalize=True), 
      disembed_shingles(embed_shingles(name, max_length, normalize=True)))

Embed Letters: 
 [11  1  9 12  9 14 27 12 21  0  0  0] kailin lu
Embed Letters Normalized: 
 [0.40740741 0.03703704 0.33333333 0.44444444 0.33333333 0.51851852
 1.         0.44444444 0.77777778 0.         0.         0.        ] kailin lu
Embed 2-Shingles: 
 [261   8 219 295 221 364 688 306   0   0   0   0] kailin lu
Embed 2-Shingles Normalized: 
 [0.35753425 0.0109589  0.3        0.40410959 0.30273973 0.49863014
 0.94246575 0.41917808 0.         0.         0.         0.        ] kailin lu


In [8]:
name = 'kailin'
max_length = 12

print('Embed Letters: \n', 
      embed_letters(name, max_length), 
      disembed_letters(embed_letters(name, max_length))) 

print('Embed Letters Normalized: \n', 
      embed_letters(name, max_length, normalize=True), 
      disembed_letters(embed_letters(name, max_length, normalize=True)))

print('Embed 2-Shingles: \n', 
      embed_shingles(name, max_length), 
      disembed_shingles(embed_shingles(name, max_length))) 

print('Embed 2-Shingles Normalized: \n', 
      embed_shingles(name, max_length, normalize=True), 
      disembed_shingles(embed_shingles(name, max_length, normalize=True)))

Embed Letters: 
 [11  1  9 12  9 14  0  0  0  0  0  0] kailin
Embed Letters Normalized: 
 [0.40740741 0.03703704 0.33333333 0.44444444 0.33333333 0.51851852
 0.         0.         0.         0.         0.         0.        ] kailin
Embed 2-Shingles: 
 [261   8 219 295 221   0   0   0   0   0   0   0] kailin
Embed 2-Shingles Normalized: 
 [0.35753425 0.0109589  0.3        0.40410959 0.30273973 0.
 0.         0.         0.         0.         0.         0.        ] kailin


## Train and save autoencoders

### Dense

In [13]:
ORIG_LENGTH = 12
BATCH_SIZE = 32
ENCODE_DIM = [256, 128] 
DECODE_DIM = [128, 256]
LR = 1e-4
EPOCHS=301
LATENT_DIM = [48]

# Embed letters 
namesA = preprocess.embed(iowa_matches['lname1915'],
                         max_length=ORIG_LENGTH, 
                         embed_type='letters', 
                         normalize=True)
namesB = preprocess.embed(iowa_matches['lname1940'],
                         max_length=ORIG_LENGTH, 
                         embed_type='letters', 
                         normalize=True)

for latent_dim in LATENT_DIM: 
    save_path = '/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/dense_letter_{}_iowa_last/'.format(latent_dim)
    run_id = 'dense_{}'.format(latent_dim)
    vae = recordlinker.model.VAE(batch_size=BATCH_SIZE,
                                 orig_dim=ORIG_LENGTH, 
                                 latent_dim=latent_dim,
                                 encode_dim=ENCODE_DIM,
                                 decode_dim=DECODE_DIM,
                                 lr=LR)

    model, encoder, decoder = vae.train(namesA, namesB, 
                                        epochs=EPOCHS, 
                                        run_id=run_id,
                                        save_path=save_path,
                                        optimizer='adam', 
                                        tensorboard=True, 
                                        earlystop=True,
                                        earlystop_patience=10,
                                        reconstruct=True, 
                                        reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
enc_0 (Dense)                   (None, 256)          3328        input_1[0][0]                    
__________________________________________________________________________________________________
enc_1 (Dense)                   (None, 128)          32896       enc_0[0][0]                      
__________________________________________________________________________________________________
mu (Dense)                      (None, 48)           6192        enc_1[0][0]                      
__________________________________________________________________________________________________
log_sigma 

In [30]:
ORIG_LENGTH = 12
BATCH_SIZE = 32
ENCODE_DIM = [128, 128] 
DECODE_DIM = [128, 128]
LR = 5e-4
EPOCHS=301
EMBED_TYPE = 'shingles'
# Embed letters 
namesA = preprocess.embed(union_matches['first1'],
                         max_length=ORIG_LENGTH, 
                         embed_type=EMBED_TYPE, 
                         normalize=True)
namesB = preprocess.embed(union_matches['first2'],
                         max_length=ORIG_LENGTH, 
                         embed_type=EMBED_TYPE, 
                         normalize=True)

LATENT_DIM = [2,4,8,16,24]
for latent_dim in LATENT_DIM: 
    save_path = '/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/dense_shingle_{}_union_first/'.format(latent_dim)
    run_id = 'dense_{}'.format(latent_dim)
    vae = recordlinker.model.VAE(batch_size=BATCH_SIZE,
                                 orig_dim=ORIG_LENGTH, 
                                 latent_dim=latent_dim,
                                 encode_dim=ENCODE_DIM,
                                 decode_dim=DECODE_DIM,
                                 lr=LR)

    model, encoder, decoder = vae.train(namesA, namesB, 
                                        epochs=EPOCHS, 
                                        run_id=run_id,
                                        save_path=save_path,
                                        optimizer='adam', 
                                        tensorboard=True, 
                                        earlystop=True,
                                        earlystop_patience=15,
                                        reconstruct=True, 
                                        reconstruct_type='s',
                                        reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
enc_0 (Dense)                   (None, 128)          1664        input_1[0][0]                    
__________________________________________________________________________________________________
enc_1 (Dense)                   (None, 128)          16512       enc_0[0][0]                      
__________________________________________________________________________________________________
mu (Dense)                      (None, 2)            258         enc_1[0][0]                      
__________________________________________________________________________________________________
log_sigma 

Epoch 36/301
3456/3456 [==============================] - 0s 77us/step - loss: 0.9072 - acc: 0.5350 - val_loss: 0.9520 - val_acc: 0.5104
Epoch 37/301
3456/3456 [==============================] - 0s 79us/step - loss: 0.9046 - acc: 0.5370 - val_loss: 0.9483 - val_acc: 0.5324
Epoch 38/301
3456/3456 [==============================] - 0s 72us/step - loss: 0.9030 - acc: 0.5391 - val_loss: 0.9372 - val_acc: 0.5301
Epoch 39/301
3456/3456 [==============================] - 0s 73us/step - loss: 0.9029 - acc: 0.5321 - val_loss: 0.9335 - val_acc: 0.5266
Epoch 40/301
3456/3456 [==============================] - 0s 72us/step - loss: 0.8996 - acc: 0.5420 - val_loss: 0.9293 - val_acc: 0.5312
Epoch 41/301
3456/3456 [==============================] - 0s 72us/step - loss: 0.8962 - acc: 0.5417 - val_loss: 0.9304 - val_acc: 0.5324
Sample Reconstructions:
{'Orig': 'ingebritson', 'Pred:': 'lqgmnsqnfcbaj'}
{'Orig': 'platt', 'Pred:': 'nrgoicaaaaaab'}
{'Orig': 'sproston', 'Pred:': 'kmkmmplgdbaag'}
{'Orig': 'pla

3456/3456 [==============================] - 0s 73us/step - loss: 1.5221 - acc: 0.4997 - val_loss: 1.3354 - val_acc: 0.5000
Epoch 3/301
3456/3456 [==============================] - 0s 71us/step - loss: 1.1713 - acc: 0.5744 - val_loss: 1.1343 - val_acc: 0.5544
Epoch 4/301
3456/3456 [==============================] - 0s 72us/step - loss: 1.0099 - acc: 0.6036 - val_loss: 0.9633 - val_acc: 0.6736
Epoch 5/301
3456/3456 [==============================] - 0s 82us/step - loss: 0.8763 - acc: 0.6861 - val_loss: 0.9009 - val_acc: 0.6852
Epoch 6/301
3456/3456 [==============================] - 0s 74us/step - loss: 0.8321 - acc: 0.6881 - val_loss: 0.8770 - val_acc: 0.6725
Epoch 7/301
3456/3456 [==============================] - 0s 72us/step - loss: 0.8037 - acc: 0.6884 - val_loss: 0.8531 - val_acc: 0.6840
Epoch 8/301
3456/3456 [==============================] - 0s 71us/step - loss: 0.7795 - acc: 0.6861 - val_loss: 0.8348 - val_acc: 0.6933
Epoch 9/301
3456/3456 [==============================] - 0s 

3456/3456 [==============================] - 0s 81us/step - loss: 1.4628 - acc: 0.6568 - val_loss: 1.2591 - val_acc: 0.6852
Epoch 3/301
3456/3456 [==============================] - 0s 85us/step - loss: 1.0369 - acc: 0.7364 - val_loss: 0.9941 - val_acc: 0.7535
Epoch 4/301
3456/3456 [==============================] - 0s 83us/step - loss: 0.8569 - acc: 0.7786 - val_loss: 0.8856 - val_acc: 0.7477
Epoch 5/301
3456/3456 [==============================] - 0s 84us/step - loss: 0.7700 - acc: 0.7775 - val_loss: 0.8210 - val_acc: 0.7419
Epoch 6/301
3456/3456 [==============================] - 0s 88us/step - loss: 0.7146 - acc: 0.7847 - val_loss: 0.7732 - val_acc: 0.7326
Epoch 7/301
3456/3456 [==============================] - 0s 91us/step - loss: 0.6808 - acc: 0.7862 - val_loss: 0.7546 - val_acc: 0.7072
Epoch 8/301
3456/3456 [==============================] - 0s 73us/step - loss: 0.6592 - acc: 0.7665 - val_loss: 0.7567 - val_acc: 0.6829
Epoch 9/301
3456/3456 [==============================] - 0s 

Epoch 53/301
3456/3456 [==============================] - 0s 74us/step - loss: 0.4803 - acc: 0.7839 - val_loss: 0.5683 - val_acc: 0.7072
Epoch 54/301
3456/3456 [==============================] - 0s 73us/step - loss: 0.4759 - acc: 0.7957 - val_loss: 0.5565 - val_acc: 0.7512
Epoch 55/301
3456/3456 [==============================] - 0s 71us/step - loss: 0.4854 - acc: 0.7847 - val_loss: 0.5765 - val_acc: 0.7083
Epoch 56/301
3456/3456 [==============================] - 0s 72us/step - loss: 0.4773 - acc: 0.7758 - val_loss: 0.5649 - val_acc: 0.7569
Epoch 57/301
3456/3456 [==============================] - 0s 72us/step - loss: 0.4762 - acc: 0.7867 - val_loss: 0.5489 - val_acc: 0.7546
Epoch 58/301
3456/3456 [==============================] - 0s 71us/step - loss: 0.4778 - acc: 0.7810 - val_loss: 0.5617 - val_acc: 0.7812
Epoch 59/301
3456/3456 [==============================] - 0s 70us/step - loss: 0.4732 - acc: 0.7882 - val_loss: 0.5500 - val_acc: 0.7593
Epoch 60/301
3456/3456 [=================

### LSTM

In [15]:
# Train 
# One hot encoding of names
ORIG_LENGTH = 12
LR = 5e-4
BATCH_SIZE = 32
EPOCHS = 350
namesA = preprocess.embed(iowa_matches['lname1915'],
                         max_length=ORIG_LENGTH, 
                         embed_type='letters', 
                         normalize=False, 
                         categorical=True)

namesB = preprocess.embed(iowa_matches['lname1940'],
                         max_length=ORIG_LENGTH, 
                         embed_type='letters',
                         normalize=False, 
                         categorical=True)


LATENT_DIM = [96,192,384]
for latent_dim in LATENT_DIM: 
    save_path = '/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/lstm_letter_{}_iowa_last/'.format(latent_dim)
    run_id = 'lstm_{}'.format(latent_dim)
    lstm_vae = recordlinker.model.LSTMVAE(batch_size=BATCH_SIZE, 
                                          timesteps=ORIG_LENGTH, 
                                          orig_dim=classes,
                                          latent_dim=latent_dim,
                                          encode_dim=[64,64], 
                                          decode_dim=[64,64],
                                          lr=LR) 
    model_lstm, model_encoder, model_decoder = lstm_vae.train(namesA, namesB, 
                                                              epochs=EPOCHS, 
                                                              run_id=run_id, 
                                                              save_path=save_path, 
                                                              earlystop=True,
                                                              earlystop_patience=10,
                                                              tensorboard=True, 
                                                              reconstruct=True, 
                                                              reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12, 28)       0                                            
__________________________________________________________________________________________________
enc_0 (LSTM)                    (None, 12, 64)       23808       input_1[0][0]                    
__________________________________________________________________________________________________
enc_1 (LSTM)                    (None, 12, 64)       33024       enc_0[0][0]                      
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 768)          0           enc_1[0][0]                      
__________________________________________________________________________________________________
mu (Dense)

3456/3456 [==============================] - 4s 1ms/step - loss: 5.4424 - acc: 0.8838 - val_loss: 7.0742 - val_acc: 0.8596
Epoch 33/350
3456/3456 [==============================] - 4s 1ms/step - loss: 5.1268 - acc: 0.8939 - val_loss: 6.9184 - val_acc: 0.8648
Epoch 34/350
3456/3456 [==============================] - 4s 1ms/step - loss: 4.9191 - acc: 0.8984 - val_loss: 6.9062 - val_acc: 0.8642
Epoch 35/350
3456/3456 [==============================] - 4s 1ms/step - loss: 4.7587 - acc: 0.9020 - val_loss: 6.7798 - val_acc: 0.8649
Epoch 36/350
3456/3456 [==============================] - 4s 1ms/step - loss: 4.6679 - acc: 0.9049 - val_loss: 6.7456 - val_acc: 0.8683
Epoch 37/350
3456/3456 [==============================] - 4s 1ms/step - loss: 4.5286 - acc: 0.9061 - val_loss: 6.6342 - val_acc: 0.8745
Epoch 38/350
3456/3456 [==============================] - 4s 1ms/step - loss: 4.2478 - acc: 0.9147 - val_loss: 6.9449 - val_acc: 0.8628
Epoch 39/350
3456/3456 [==============================] - 4s 

3456/3456 [==============================] - 4s 1ms/step - loss: 15.3422 - acc: 0.6206 - val_loss: 15.0229 - val_acc: 0.6315
Epoch 8/350
3456/3456 [==============================] - 4s 1ms/step - loss: 14.4439 - acc: 0.6422 - val_loss: 14.1916 - val_acc: 0.6592
Epoch 9/350
3456/3456 [==============================] - 4s 1ms/step - loss: 13.6969 - acc: 0.6623 - val_loss: 13.4984 - val_acc: 0.6715
Epoch 10/350
3456/3456 [==============================] - 5s 1ms/step - loss: 12.9873 - acc: 0.6772 - val_loss: 12.9910 - val_acc: 0.6799
Epoch 11/350
3456/3456 [==============================] - 5s 2ms/step - loss: 12.3171 - acc: 0.6943 - val_loss: 12.3037 - val_acc: 0.6942
Sample Reconstructions:
{'Orig': 'fuller', 'Pred:': 'mlller'}
{'Orig': 'koepke', 'Pred:': 'moppee'}
{'Orig': 'hill', 'Pred:': 'llll'}
{'Orig': 'innes', 'Pred:': 'innes'}
{'Orig': 'koenig', 'Pred:': 'munnng'}
Epoch 12/350
3456/3456 [==============================] - 5s 1ms/step - loss: 11.5995 - acc: 0.7119 - val_loss: 11.78

Train on 3456 samples, validate on 864 samples
Epoch 1/350
3456/3456 [==============================] - 6s 2ms/step - loss: 24.6410 - acc: 0.4874 - val_loss: 20.5443 - val_acc: 0.5203
Sample Reconstructions:
{'Orig': 'stahley', 'Pred:': 'eeeeenn'}
{'Orig': 'hilf', 'Pred:': 'eeee'}
{'Orig': 'nigg', 'Pred:': 'eeee'}
{'Orig': 'peoppe', 'Pred:': 'eeeeen'}
{'Orig': 'mcgraw', 'Pred:': 'eeeeenn'}
Epoch 2/350
3456/3456 [==============================] - 5s 1ms/step - loss: 19.3109 - acc: 0.5353 - val_loss: 18.6825 - val_acc: 0.5507
Epoch 3/350
3456/3456 [==============================] - 5s 1ms/step - loss: 18.1067 - acc: 0.5575 - val_loss: 17.6537 - val_acc: 0.5660
Epoch 4/350
3456/3456 [==============================] - 5s 1ms/step - loss: 16.9827 - acc: 0.5771 - val_loss: 16.6869 - val_acc: 0.5819
Epoch 5/350
3456/3456 [==============================] - 5s 1ms/step - loss: 16.1437 - acc: 0.5918 - val_loss: 15.9201 - val_acc: 0.6024
Epoch 6/350
3456/3456 [==============================] - 4s

Saved decoder in: /Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/lstm_letter_384_iowa_last/decoder.h5
